<a href="https://colab.research.google.com/github/DeepManuPy/convNets/blob/master/StanfordDogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Info on how to get your api key (kaggle.json) here: https://github.com/Kaggle/kaggle-api#api-credentials

#!pip install -q kaggle
import json
import zipfile
import os
api_token = {"username":"reddymanudeep","key":"882ba15e574c727d4ef37d746c1d9044"}
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)


In [0]:
!cd /root && ls -a

In [0]:
!pip install tensorflow-gpu

In [0]:
!pip install split-folders

In [0]:
!kaggle datasets download -d jessicali9530/stanford-dogs-dataset

In [0]:
!unzip stanford-dogs-dataset.zip
!tar xf images.tar
!tar xf annotations.tar

In [0]:
import os
classes = list(os.listdir("Images"))
classes.sort()
print(classes)

In [0]:
from os import mkdir
mkdir("cropped_images")
for name in classes:
  os.mkdir(os.path.sep.join(["cropped_images",name]))

In [0]:
from PIL import Image
from xml.etree import ElementTree

for classdir in os.listdir("Annotation"):
  subdir = os.path.sep.join(["Annotation",classdir])
  subimg = os.path.sep.join(["Images",classdir])
  for filename in os.listdir(subdir):
    
    tree = ElementTree.parse(os.path.sep.join([subdir,filename]))
    root = tree.getroot()
    box = root.find(".//bndbox")
    xmin = int(box.find("xmin").text)
    ymin = int(box.find("ymin").text)
    xmax = int(box.find("xmax").text)
    ymax = int(box.find("ymax").text)
    
    file = filename+".jpg"
    img = os.path.sep.join([subimg,file])
    image = Image.open(img)
    image = image.crop((xmin,ymin,xmax,ymax))
    n = img.split(os.path.sep)
    file = filename+".png"
    image.save(os.path.sep.join(["cropped_images",classdir,file]))
  

In [0]:
import split_folders

split_folders.ratio("cropped_images",output="dataset",seed=1337,ratio=(.8,.1,.1))

In [0]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np


train_iter = ImageDataGenerator(rotation_range=30,
                               zoom_range=0.15,
                               width_shift_range=0.2,
                              height_shift_range=0.2,
                              shear_range=0.15,
                              horizontal_flip=True,
                              fill_mode="nearest")

#train_iter = ImageDataGenerator()
val_iter = ImageDataGenerator()
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
train_iter.mean = mean
val_iter.mean = mean

train_gen = train_iter.flow_from_directory("dataset/train",
                                           class_mode="categorical",
                                          target_size = (224,224),
                                          shuffle = True,
                                          batch_size = 32)
val_gen = val_iter.flow_from_directory("dataset/val",
                                           class_mode="categorical",
                                          target_size = (224,224),
                                          shuffle = True,
                                          batch_size = 32)

test_gen = val_iter.flow_from_directory("dataset/test",
                                           class_mode="categorical",
                                          target_size = (224,224),
                                          shuffle = True,
                                          batch_size = 32)


Using TensorFlow backend.


Found 16418 images belonging to 120 classes.
Found 2009 images belonging to 120 classes.
Found 2153 images belonging to 120 classes.


In [0]:
from keras.models import Model
from keras.layers import Flatten,Dense,Dropout,BatchNormalization,Activation
from keras.applications.vgg16 import VGG16,decode_predictions
from keras.models import Model

base_model = VGG16(include_top=False,input_shape=(224,224,3),weights="imagenet")
flat1 = Flatten()(base_model.output)
#drop = Dropout(0.4)(flat1)
fc = Dense(2048,kernel_initializer="he_normal")(flat1)
#drop = Dropout(0.5)(fc)
bn = BatchNormalization()(fc)
act1 = Activation("relu")(bn)
fc1 = Dense(1024,kernel_initializer="he_normal")(act1)
#drop1 = Dropout(0.5)(fc1)
bn = BatchNormalization()(fc1)
act2 = Activation("relu")(bn)
out = Dense(120,activation='softmax')(act2)


model = Model(inputs=base_model.inputs,outputs=out)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [0]:
from keras.optimizers import adam,sgd
from keras.callbacks import *

ad = adam(lr = 1e-2)
for layers in base_model.layers:
  layers.trainable = False

model.compile(loss='categorical_crossentropy',optimizer=ad,metrics=['accuracy'])
model.load_weights("weights_1.489.hdf5")
his = model.fit_generator(train_gen,epochs=5,steps_per_epoch=16670//32,verbose=1,
                    validation_data=val_gen,validation_steps=2041//32,callbacks=[
                        ModelCheckpoint("weights_{val_loss:.3f}.hdf5",save_best_only=True,
                                       save_weights_only=True,verbose=0,mode="min")
                    ])


In [0]:
from matplotlib import pyplot as plt
def plot_training(H, N, plotPath):
	# construct a plot that plots and saves the training history
	plt.style.use("ggplot")
	plt.figure()
	plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
	plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
	plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
	plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
	plt.title("Training Loss and Accuracy")
	plt.xlabel("Epoch #")
	plt.ylabel("Loss/Accuracy")
	plt.legend(loc="lower left")
	plt.savefig(plotPath)

In [0]:
plot_training(his,10,"WarmUp.png")

In [0]:
from keras.preprocessing.image import load_img,img_to_array
image = load_img("dog.jpg",target_size=(224,224))
image = img_to_array(image)
image = image/225.0
image = image.reshape(1,224,224,3)

results = model.predict(image)
print(results)

In [0]:
train_gen.reset()
val_gen.reset()

for layer in base_model.layers[15:]:
  layers.trainable = True

model.load_weights("weights.1.38.hdf5")
opt = adam(lr=1e-6)
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
his = model.fit_generator(train_gen,epochs=20,steps_per_epoch=16670//32,verbose=1,
                    validation_data=val_gen,validation_steps=2041//32,callbacks=[
                        ModelCheckpoint("weights-{val_loss:.2f}.hdf5",save_best_only=True,
                                       save_weights_only=True,verbose=0)
                    ])


  